<a href="https://colab.research.google.com/github/BenxiaHu/DeepLearning/blob/main/pytorch_1D_CNN_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network


Dependencies:
* torch: 0.1.11
* matplotlib

# step0: import python packages

In [35]:
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.nn.functional as Fun
from numpy import array
import pandas as pd
import numpy as np
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import requests
from torch.nn import LogSoftmax
from torch.optim.lr_scheduler import ReduceLROnPlateau
torch.manual_seed(1)



# step1: import data

In [36]:
seqid="https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/K562_MPRA_Silencers.txt"
#print(seqid)
seqid = pd.read_csv(seqid,header=0,sep="\t")
seqid["peakid"] = seqid['chr'] +":"+ seqid["start"].astype(str) +"-"+ seqid["end"].astype(str)
#print(seqid)
seqid = seqid[["peakid","silencer"]]
#print(seqid)

fastaid="https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/K562_MPRA_Silencers.fasta"
fastaid = pd.read_table(fastaid,header=None,sep="\t")
#print(fastaid)
fastaid.rename(columns={0:"peakid",1:"fasta"},inplace=True)

result = pd.merge(fastaid, seqid, how='inner', on=['peakid', 'peakid'])

result[["fasta"]] = result[["fasta"]].apply(lambda x: x.astype(str).str.upper())

result = result[["fasta","silencer"]]

#label="https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/labels.txt"
#label = pd.read_table(labelid,header=None)
input = result[["fasta"]]
#print(input.shape)
label = result[["silencer"]]

# Convert input to one-hot code
#def one_hot_encoding(seq):
#    mapping = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1]}
#    one_hot = [mapping[nuc] for nuc in seq]
#    return np.array(one_hot)

#one_hot_input = np.array([one_hot_encoding(seq) for seq in input[0]])
#
#one_hot_input = torch.from_numpy(one_hot_input)
#one_hot_input = one_hot_input.permute(0, 2, 1)
#print(one_hot_input.shape)

# Split the data into training, test, and prediction sets
# Split the data into training, testing, and prediction sets
#x_train, x_test, y_train, y_test = train_test_split(one_hot_input, label, test_size=0.3, random_state=42)
#x_test, x_pred, y_test, y_pred = train_test_split(x_test, y_test, test_size=0.67, random_state=42)
#print(x_train.shape)
#print(x_test.shape)


DNA = np.zeros(shape=(len(input),len(input["fasta"][0]),4))
labelid = np.zeros(shape=(len(input),))
#print(DNA.shape)
#print(labelid.shape)

for i in range(input.shape[0]):
    seq_array = array(list(input["fasta"][i]))
    #integer encode the sequence
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    #one hot the sequence
    onehot_encoder = OneHotEncoder(sparse_output=False)
    #reshape because that's what OneHotEncoder likes
    #print(integer_encoded_seq.shape)
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    #print(integer_encoded_seq.shape)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)
    #print(onehot_encoded_seq.shape)
    #print(len(onehot_encoded_seq))
    DNA[i] = onehot_encoded_seq
    #DNA[i] = onehot_encoded_seq.reshape(4,len(onehot_encoded_seq))
    labelid[i] = label["silencer"][i]

#print(DNA.shape)
#print(labelid.shape)

DNA = torch.tensor(DNA)
DNA = DNA.permute(0, 2, 1)
labelid =  torch.tensor(labelid)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    DNA = DNA.to("cuda:0")
    labelid =  labelid.to("cuda:0")
#print(DNA.is_cuda)

#print(np.shape(DNA))
#embed_x = embed_x.permute(0, 2, 1)

# step2: split the data into training, test and prediction data

In [37]:
#input_tensor = DNA
#label_tensor = labelid
# pick 1400 samples as training data
#print(input_tensor[0:1400,:,:].shape)
#print(label_tensor[0:1400].shape)

# Split the data into training, test, and prediction sets
x_train, x_test, y_train, y_test = train_test_split(DNA, labelid, test_size=0.3, random_state=42)

#print(x_train.shape)
# Hyper Parameters
BATCH_SIZE = 16
torch_dataset = Data.TensorDataset(x_train, y_train)
train_loader = Data.DataLoader(dataset=torch_dataset, batch_size=BATCH_SIZE, shuffle=True)
torch_dataset2 = Data.TensorDataset(x_test, y_test)
test_loader = Data.DataLoader(dataset=torch_dataset2, shuffle=True)
              
# Data Loader for easy mini-batch return in training, the image batch shape will be (100, 1, 50, 4)

#for i, j in enumerate(train_loader):
    #x, y = j
    #print('batch:{0} x:{1}  y: {2}'.format(i, x, y))
    #print(i)
    #print(x.shape)
    #print(y.shape)

# pick 400 samples as prediction data
#test_x2 = Variable(input_tensor[1600:2000,:,:]).type(torch.FloatTensor)
#test_y2 = label_tensor[1600:2000]


# step4: build the 1D-CNN model

In [40]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv1d(in_channels=4, out_channels=64, 
                                             kernel_size=3, padding=1),
                     nn.BatchNorm1d(num_features=64),
                     nn.ReLU(),
                     nn.MaxPool1d(kernel_size=2))
        self.conv2 = nn.Sequential(nn.Conv1d(in_channels=64, out_channels=128, 
                                             kernel_size=3, padding=1),
                     nn.BatchNorm1d(num_features=128),
                     nn.ReLU(),
                     nn.MaxPool1d(kernel_size=2))
        self.conv3 = nn.Sequential(nn.Conv1d(in_channels=128, out_channels=256, 
                                             kernel_size=3, padding=1),
                     nn.BatchNorm1d(num_features=256),
                     nn.ReLU(),
                     nn.MaxPool1d(kernel_size=2))
        self.fc1 = nn.Linear(256*25, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        out = self.fc3(x)
        return out              # return x for visualization



cnn = CNN()
#print(cnn)  # net architecture
LR = 0.00001 
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

# running the CNN model

In [41]:
# training and testing
EPOCH = 3000              # train the training data n times, to save time, we just train 1 epoch

optimizer.zero_grad()           # clear gradients for this training step
train_losses = []
trainCorrect = 0
for epoch in range(EPOCH):
    running_loss = 0.0
    j=0
    for i, (b_x, b_y) in enumerate(train_loader):  # gives batch data, normalize x when iterate train_loader
        #print(b_x.shape)
        b_x = Variable(b_x).type(torch.FloatTensor)
        b_y = b_y.type(torch.FloatTensor)
        optimizer.zero_grad()           # clear gradients for this training step
        output = cnn(b_x)               # cnn output
        preds, predsid = torch.max(output,1) 
        loss = loss_func(preds, b_y)   # cross entropy loss
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        # print statistics
        running_loss += loss.item()
        j+=1

    print(f'the loss on each epoch: {epoch}, loss: {running_loss/j}')
    train_losses.append(running_loss/j)
print('Finished Training')
#trainSteps = len(train_loader.dataset) // BATCH_SIZE
# calculate the average training and validation loss
#avgTrainLoss = sum(train_losses) / trainSteps
# calculate the training and validation accuracy
#trainCorrect = trainCorrect / trainSteps

# Let’s quickly save our trained model:
# Plot the training and test losses over time
plt.plot(train_losses, label='Train Loss')
plt.legend()
plt.xlabel("# of epoch")
plt.ylabel("loss")
plt.show()
!pwd
torch.save(cnn.state_dict(), "./content")



torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([16, 256, 25])
torch.Size([16, 64, 100])
torch.Size([16, 128, 50])
torch.Size([

KeyboardInterrupt: ignored

# apply the model to the test data

In [ ]:
net = CNN()
net.load_state_dict(torch.load("./content"))
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, (b_x, b_y) in enumerate(test_loader):
        b_x = Variable(b_x).type(torch.FloatTensor)
        #print(b_x.shape)
        #b_y = b_y.type(torch.LongTensor)
        # calculate outputs by running the network
        outputs = net(b_x)
        # the class with the highest energy is what we choose as prediction
        preds, predsid = torch.max(outputs,1)
        total += b_y.size(0)
        correct += (predsid == b_y).sum().item()           

print(f'Accuracy of the network on the 10000 test : {100 * correct // total} %')
